# Uczenie_transferowe_w_Keras

Ten notatnik pokazuje, jak załadować przetrenowany model (w tym przypadku VGGNet19) i dotroić go do nowego zadania: rozpoznawania hot dogów.

#### Załadowanie zależności.


In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


#### Załadowanie przetrenowanego modeluVGGNet19.

In [ ]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Zamrożenie wszystkich warstw w podstawowym modelu VGGNet19.

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

#### Dodanie niestandardowych warstw klasyfikacyjnych.

In [ ]:
# Utworzenie instancji modelu sekwencyjnego i dodanie modelu VGG19.
model = Sequential()
model.add(vgg19)

# Dodanie niestandardowych warstw do modelu VGG19.
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Kompilacja modelu przed treningiem.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Przygotowanie danych do treningu.

Zbiór danych można pobrać [stąd](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).

In [ ]:
## Uwaga, od 2019-11-01 powyższy link nie działa. Zamiast tego należy usunąć
## komentarze z poniższego kodu ładującego dane:
# ! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
# ! tar -xzf hot-dog-not-hot-dog.tar.gz

In [ ]:
# Utworzenie instancji dwóch klas generatorów obrazów.
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [ ]:
# Definicja wielkości paczki.
batch_size=32

In [ ]:
# Definicje generatorów danych treningowych i weryfikacyjnych.
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 43s - loss: 0.9661 - acc: 0.5729 - val_loss: 0.5618 - val_acc: 0.7146
Epoch 2/16
15/15 [==============================] - 5s - loss: 0.7775 - acc: 0.6486 - val_loss: 0.5116 - val_acc: 0.7393
Epoch 3/16
15/15 [==============================] - 4s - loss: 0.5765 - acc: 0.7410 - val_loss: 0.6632 - val_acc: 0.6966
Epoch 4/16
15/15 [==============================] - 4s - loss: 0.4318 - acc: 0.8123 - val_loss: 0.4298 - val_acc: 0.7949
Epoch 5/16
15/15 [==============================] - 4s - loss: 0.3937 - acc: 0.8076 - val_loss: 0.4536 - val_acc: 0.7885
Epoch 6/16
15/15 [==============================] - 4s - loss: 0.3500 - acc: 0.8524 - val_loss: 0.4060 - val_acc: 0.8120
Epoch 7/16
15/15 [==============================] - 4s - loss: 0.3216 - acc: 0.8514 - val_loss: 0.5474 - val_acc: 0.7438
Epoch 8/16
15/15 [==============================] - 4s - loss: 0.2992 - acc: 0.8571 - val_loss: 0.4914 - val_acc: 0.7714
Epoch 9/16
15/15 [=============